In [1]:
pip install gradio plotly pandas numpy scikit-learn requests

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.express as px
import gradio as gr

# KONFIGURACJA
BASE = "https://bdl.stat.gov.pl/api/v1"

MY_API_KEY = "d4a9b4b9-20f6-46cd-ae1e-08de56c2a7a2" 

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'X-ClientId': MY_API_KEY
}

INDICATORS = {
    "Powierzchnia_terenow_chronionych": ["1540"],
    "Zielen_parki": ["73847", "73849", "73848"],
    "Miejsca_noclegowe": ["40936"],
    "Gastronomia_ogolem": ["64542", "64543", "64544", "64545"],
    "Liczba_imprez": ["377328"],
    "Dlugosc_komunikacji_miejskiej": ["400294"],
    "Dlugosc_drog_twardych": ["54574"],
    "Dlugosc_linii_kolejowych": ["4737"],
    "Liczba_przestepstw": ["58559"],
    "Populacja": ["72305"],
    "Powierzchnia": ["1"]
}

# GeoJSON - mapa Polski
geojson_url = "https://raw.githubusercontent.com/ppatrzyk/polska-geojson/master/wojewodztwa/wojewodztwa-max.geojson"
try:
    geojson_poland = requests.get(geojson_url).json()
except:
    geojson_poland = {}
    print("Błąd pobierania GeoJSON")

def fetch_bulk_data(year):
    """
    Pobiera dane dla wszystkich województw (unit-level=2) zmienna po zmiennej.
    """
    data_store = {} 
    print(f"Rozpoczynam pobieranie danych za rok {year}...")

    for indicator_name, var_ids in INDICATORS.items():
        print(f"Pobieranie wskaźnika: {indicator_name}")
        
        for vid in var_ids:
            url = f"{BASE}/data/by-variable/{vid}?unit-level=2&year={year}&format=json&page-size=100"
            success = False
            for _ in range(5): # Retry logic
                try:
                    r = requests.get(url, headers=HEADERS, timeout=30)
                    if r.status_code == 200:
                        data = r.json()
                        results = data.get("results", [])
                        
                        for item in results:
                            unit_id = item["id"]
                            unit_name = item["name"]
                            val = item["values"][0]["val"] if item.get("values") else 0
                            val = float(val) if val is not None else 0.0

                            if unit_id not in data_store:
                                data_store[unit_id] = {"Województwo": unit_name}
                                for k in INDICATORS.keys():
                                    data_store[unit_id][k] = 0.0
                            
                            data_store[unit_id][indicator_name] += val
                        
                        success = True
                        break 
                    elif r.status_code == 429:
                        print("Limit zapytań (429), czekam 10s...")
                        time.sleep(10)
                    else:
                        print(f"Błąd HTTP {r.status_code} dla {vid}")
                        time.sleep(2)
                except Exception as e:
                    print(f"Wyjątek: {e}, czekam 5s...")
                    time.sleep(5)
            
            if not success:
                print(f"Nie udało się pobrać danych dla zmiennej {vid}")
            
            time.sleep(2.0) 

    return pd.DataFrame(data_store.values())

def analyze_tourism(year: str):
    year = str(year).strip() or "2022"
    
    # 1. POBIERANIE DANYCH
    df = fetch_bulk_data(year)
    
    if df.empty:
        return pd.DataFrame(), None

    df_display = df.copy()

    # 2. PRZETWARZANIE DANYCH
    df["Powierzchnia"] = df["Powierzchnia"].replace(0, 1) 
    df["Populacja"] = df["Populacja"].replace(0, 1)

    df["X1"] = df["Powierzchnia_terenow_chronionych"] / df["Powierzchnia"]
    df["X2"] = df["Zielen_parki"] / df["Powierzchnia"]
    df["X4"] = df["Miejsca_noclegowe"] / df["Populacja"] * 1000
    df["X6"] = df["Gastronomia_ogolem"] / df["Populacja"] * 10000
    df["X7"] = df["Liczba_imprez"] / 365
    df["X10"] = df["Dlugosc_komunikacji_miejskiej"] / df["Populacja"] * 10000
    df["X11"] = df["Dlugosc_drog_twardych"] / df["Populacja"] * 1000
    df["X12"] = df["Dlugosc_linii_kolejowych"] / df["Populacja"] * 10000
    
    sum_przestepstw = df["Liczba_przestepstw"].sum()
    if sum_przestepstw == 0: sum_przestepstw = 1
    df["X15"] = - (df["Liczba_przestepstw"] / sum_przestepstw * 100)

    cols = ["X1","X2","X4","X6","X7","X10","X11","X12","X15"]
    df[cols] = df[cols].replace([np.inf, -np.inf], np.nan).fillna(0)

    # 3. PCA
    X = StandardScaler().fit_transform(df[cols])
    pca = PCA().fit(X)
    scores = pca.transform(X)
    
    if sum(pca.components_[0][i] for i in range(len(cols)-1)) < 0:
        scores[:,0] = -scores[:,0]

    df_map = pd.DataFrame({"Województwo": df["Województwo"], "PC1": scores[:,0]})
    

    df_map["geo"] = df_map["Województwo"].str.lower()

    # 4. RYSOWANIE MAPY
    fig = px.choropleth(df_map, 
                        geojson=geojson_poland, 
                        locations="geo", 
                        featureidkey="properties.nazwa",
                        color="PC1", 
                        color_continuous_scale="RdYlGn", 
                        range_color=(df_map["PC1"].min(), df_map["PC1"].max()),
                        hover_name="Województwo", 
                        hover_data={"PC1": ":.3f", "geo": False}) # geo:False ukrywa klucz w dymku
    
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(title=f"Atrakcyjność turystyczna województw Polski ({year})",
                      margin={"l":0,"r":0,"t":50,"b":0}, height=600)
    
    return df_display, fig

# INTERFEJS GRADIO
with gr.Blocks() as demo:
    gr.Markdown("## Analiza Atrakcyjności Turystycznej Województw (GUS)")
    
    with gr.Row():
        year_input = gr.Textbox(label="Rok danych", value="2022")
        btn = gr.Button("Pobierz dane i generuj mapę")
    
    gr.Markdown("### Pobrane dane surowe")
    data_output = gr.Dataframe(label="Tabela Danych", interactive=False)
    
    gr.Markdown("### Mapa Wynikowa (PCA)")
    plot_output = gr.Plot()

    btn.click(fn=analyze_tourism, inputs=year_input, outputs=[data_output, plot_output])

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.


Rozpoczynam pobieranie danych za rok 2022...
Pobieranie wskaźnika: Powierzchnia_terenow_chronionych
Pobieranie wskaźnika: Zielen_parki
Pobieranie wskaźnika: Miejsca_noclegowe
Pobieranie wskaźnika: Gastronomia_ogolem
Pobieranie wskaźnika: Liczba_imprez
Pobieranie wskaźnika: Dlugosc_komunikacji_miejskiej
Pobieranie wskaźnika: Dlugosc_drog_twardych
Pobieranie wskaźnika: Dlugosc_linii_kolejowych
Pobieranie wskaźnika: Liczba_przestepstw
Pobieranie wskaźnika: Populacja
Pobieranie wskaźnika: Powierzchnia
